$$\textbf{Optimización I. Tarea 3}$$
$$\textit{Y. Sarahi García Gozález}$$

$\color{lightblue}{Librerías \space }$ <br>

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
from math import sqrt

In [2]:
print("Tarea realizada en MacOs. \nLas versiones de las librerías y de python utilizadas fueron:\n")
from platform import python_version
print("Python version", python_version())
print("Numpy version", np.__version__)

Tarea realizada en MacOs. 
Las versiones de las librerías y de python utilizadas fueron:

Python version 3.11.7
Numpy version 1.26.3


In [8]:
#imprimimos el epsilon de la máquina
epsilon = np.finfo(float).eps
print("Epsilon de la máquina:", epsilon)

Epsilon de la máquina: 2.220446049250313e-16


$\color{lightblue}{Ejercicio \space 1}$ <br>

1. Programar la función que implementa el algoritmo de backtracking 
   (Algoritmo 2 de la Clase 6) que usa la condición de descenso suficiente
   (condición de Armijo) para seleccionar el tamaño de paso.

In [39]:
def backtracking(alpha_ini,rho,c,x_k,f,f_k,df_k,p_k,iter_max):

    '''
        Esta funcion parte de un tamaño de paso inicial alpha_ini y lo va recortando hasta que
        cumple la cond de descenso suficiente

        parametros:
            valores (float): alpha_ini, rho entre (0,1), f(x_k), Df(x_k) (gradiente en el punto x_k), c_1, 
            direccion de descenso (np.rray): p_k 

        returns:
            el tamaño de paso a_k
            numero de iteraciones realizadas i_k
    '''

    alpha=alpha_ini #fijamos alpha como el alpha inicial
    gp=c*np.dot(df_k,p_k) #hacemos el producto gradiente por direccion de descenso p
    
    for i in range(iter_max):
        x_kp=x_k+alpha*p_k

        #si la condicion de descenso se cumple, terminamos
        if f(x_kp)<=(f_k + alpha*gp):
            return alpha,i,True

        alpha=alpha*rho #si no se cumple la cond, hacemos alpha*rho
        print(alpha)

        
    return  alpha,i,False

2. Programar la función que implementa el algoritmo de descenso máximo con
   backtracking

- La función $f(\mathbf{x})$, 
- el gradiente $\nabla f(\mathbf{x})$ de la función $f$, 
- un punto inicial $\mathbf{x}_{0}$, 
- las tolerancia $\tau>0$, 
- el número máximo de iteraciones $N$ para el algoritmo de descenso máximo, 
- el valor inicial $\alpha_{ini}$,
- el valor $\rho \in (0,1)$,
- la constante $c_1 \in (0,1)$ para la condición de Armijo y
- el número máximo de iteraciones $N_{gs}$ para el método de la sección dorada.

La función devuelve 
- El último punto $\mathbf{x}_{k}$ generado por el algoritmo,
- el número $k$ de iteraciones realizadas y
- Una variable indicadora que es $True$ si el algoritmo termina por 
  cumplirse la condición de paro ($\|\alpha_k \mathbf{p}_{k}\| < \tau$) o
  $False$ si termina porque se alcanzó el número máximo de iteraciones.
- Si $n\ne 2$, devuelve un arreglo vació. En caso contrario, devuelve un 
  arreglo que contiene las componentes de los puntos de la secuencia, el
  tamaño de paso y la cantidad de iteraciones que hizo el algoritmo 
  de backtracking en cada iteración:

In [69]:

def descenso_maximo_backtracking(f,df,x0,alpha_ini,rho,c,tau,max_iter,max_iter_b):
    '''
    Esta funcion busca el minimo de la funcion f usando la tecnica de backtracking

    Parametros: 
        f: función a optimizar
        df: gradiente de f
        x_0: valor inicial
        tau,N: tolerancia y numero maximo de iteraciones (descenso)
        parametros de 

        NOTA:los argumentos predeterminados son específicos para este ejercicio y n=2
    
    returns:
        x_k: ultimo punto de la sucesión que genera el algoritmo
        k: número de iteraciones
        True/False: Indica si se satisfizo la condición de tolerancia
        x1,x2...xk: sucesión de puntos (np.array)
        
    '''
    n=len(x0)
    x_k=x0
    indicador=False
    f_k=f(x_k)
    df_k=df(x_k)


    if n==2:
        m = np.zeros((max_iter+1,4))
        m[0,:] = x0[0],x0[1],1,0
    
 
    for k in range(max_iter):
        #calculmos pk y ak
        p_k=-df_k
        a_k,i,ind=backtracking(alpha_ini,rho,c,x_k,f,f_k,df_k,p_k,max_iter_b)
   
        if not ind:
            print('Insuficientes iteraciones Backtracking', a_k)

        x_k=x_k+(a_k*p_k)
        
        if n==2:
            m[k,:]= x_k[0],x_k[1],a_k,i


        if norm(a_k*p_k)<tau:#si se cumple la condicion de tolerania:
            indicador=True #indicador verdadero
            break #y romepos el ciclo

        f_k=f(x_k)
        df_k=df(x_k)

        
    if n==2:
        return x_k,k,indicador,m
    
    return x_k,k,indicador,None

3. Para probar el algoritmo, programe las siguientes funciones, calcule su gradiente 
   de manera analítica y programe la función correspondiente. Use cada punto 
   $\mathbf{x}_0$ como punto inicial del algoritmo.  


In [74]:
tau = sqrt(epsilon) #tolerancia (después multiplicaremos por sqrt(n))
aini = 1.0 
rho  = 0.8 
c1   = 0.1
N    = 30000 #iter maximas para descenso
Nb   = 600 #iter maximas para backtracking

**Función de Himmelblau:** Para $\mathbf{x}=(x_1,x_2)$

$$ f(\mathbf{x}) = (x_1^2 + x_2 - 11)^2 + (x_1 + x_2^2 - 7)^2. $$
$$ \mathbf{x}_0 = (2.,4.) $$
$$ \mathbf{x}_0 = (0.,0.) $$

In [75]:
# Función objetivo
def Himmelblau(x):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2;

# Gradiente de la función objetivo
def D_Himmelblau(x):
    gx = 4*x[0]*(x[0]**2 + x[1] - 11) + 2*(x[0] + x[1]**2 - 7)
    gy = 2*(x[0]**2 + x[1] - 11) + 4*x[1]*(x[0] + x[1]**2 - 7)
    return np.array([gx, gy])


En cada caso imprima los resultados:
- El número de iteraciones realizadas $k$
- El punto $\mathbf{x}_k$ obtenido
- $f(\mathbf{x}_k)$
- $\|\nabla f(\mathbf{x}_k)\|$
- La variable que indica si el algoritmo terminó porque se cumplió el criterio de paro o no.
Además, si $n=2$, imprima
- El valor promedio de los tamaños de paso $\alpha_0, \alpha_1, ..., \alpha_k$.
- El valor promedio de las iteraciones $i_0, i_1, ..., i_k$ realizadas por el algoritmo de backtracking.
- La gráfica de los contornos de nivel de la función y la trayectoria
  de los puntos $\mathbf{x}_0, \mathbf{x}_1, ..., \mathbf{x}_k$.



In [76]:
x0  = np.array([2,4])
n   = len(x0)
tau = np.sqrt(n) * tau

xk, k, indicador, m = descenso_maximo_backtracking(Himmelblau,D_Himmelblau,x0,aini,rho,c1,tau,N,Nb)

print('Num.Iter = ', k)
print('xk       = ', xk)




Num.Iter =  29
xk       =  [3. 2.]


In [28]:
m

array([[2.00000000e+00, 4.00000000e+00, 6.03833988e-19, 1.88000000e+02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

**Función de Beale :** Para $\mathbf{x}=(x_1,x_2)$

$$ f(\mathbf{x}) = (1.5-x_1 + x_1x_2)^2 + (2.25 - x_1 + x_1x_2^2)^2 + (2.625 - x_1 + x_1x_2^3)^2.$$
$$ \mathbf{x}_0 = (2.,3.) $$
$$ \mathbf{x}_0 = (2.,4.) $$
   



In [4]:
# Definir la función f
def Beale(x):
    return (1.5 - (x[0]) + (x[0])*(x[1]))**2 + (2.25 - (x[0]) + (x[0])*(x[1])**2)**2 + (2.625 - (x[0]) + (x[0])*(x[1])**3)**2

def D_Beale(x):
    Partial_Beale_dx1 = 2*(1.5 - (x[0]) + (x[0])*(x[1]))*(-1 + (x[1])) + 2*(2.25 - (x[0]) + (x[0])*(x[1])**2)*(-1 + (x[1])**2) + 2*(2.625 - (x[0]) + (x[0])*(x[1])**3)*(-1 + (x[1])**3)
    Partial_Beale_dx2 = 2*(1.5 - (x[0]) + (x[0])*(x[1]))*((x[0])) + 2*(2.25 - (x[0]) + (x[0])*(x[1])**2)*(2*(x[0])*(x[1])) + 2*(2.625 - (x[0]) + (x[0])*(x[1])**3)*(3*(x[0])*(x[1])**2)
    return np.array([Partial_Beale_dx1, Partial_Beale_dx2])


**Función de Rosenbrock:** Para $\mathbf{x}=(x_1,x_2, ..., x_n)$

$$ f(\mathbf{x}) = \sum_{i=1}^{n-1} \left[100(x_{i+1} - x_i^2)^2 + (1-x_i)^2 \right]
\quad n\geq 2.$$
$$ \mathbf{x}_0 = (-2.1, 4.5) $$
$$ \mathbf{x}_0 = (-1.2, 1.0) $$
$$ \mathbf{x}_0 = (-2.1, 4.5, -2.1, 4.5, -2.1, 4.5, -2.1, 4.5, -2.1, 4.5) $$
$$ \mathbf{x}_0 = (-1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0) $$

In [5]:
def Rosenbrock(x):
    n = len(x)
    suma = 0
    for i in range(n-1):
        suma += 100 * (x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return suma

def D_Rosenbrock(x):
    n = len(x)
    gradient = np.zeros(n)
    for i in range(n-1):
        gradient[i] += -400 * x[i] * (x[i+1] - x[i]**2) - 2 * (1 - x[i])
        gradient[i+1] += 200 * (x[i+1] - x[i]**2)
    return gradient

3. Nocedal sugiere que la constante $c_1$ sea del orden de $0.0001$. 
   Use $c_1=0.0001$ y  repeta la prueba con la función de Beale y 
   explique en qué casos conviene usar un valor grande o pequeño de $c_1$.

$\color{lightblue}{Ejercicio \space 2}$ <br>

$\color{lightblue}{Ejercicio \space 3}$ <br>

3.1 Verificamos si $p_0$ es dirección de descenso:

* El gradiente de f:
\begin{align*}
f(x_1, x_2) &= 5 + (x_1)^2 + (x_2)^2 \\
\nabla f(x_1, x_2) &= \nabla f{x_1}{x_2} \\
&= (2x_1, 2x_2)
\end{align*}

\begin{align*}
\implies \nabla f(\textbf{x}_0)= (-2, 2)
\end{align*}

* Dirección p:

\begin{align*}
\textbf{p}_0=(1,0) 
\end{align*}


\begin{align*}
&\implies \nabla f(\textbf{x}_0)\textbf{p}_0=(-2,2)^T (1,0)=-2+0=-2\\
\end{align*}

$$\implies \nabla f(\textbf{x}_0)\textbf{p}_0 <0$$

Por lo que $p_0$ es dirección de descenso


3.2 Encontramos $\alpha_{max}$ que satisface la dirección de descenso:


* f evaluada en $x_0 + p_0$ 
$$f(x_0 + p_0)=f((-1,1)+(1,0))=f(0,1)=6$$

* f evaluada en $x_0$

$$f(x_0)=f(-1,1)=7$$

*Establecemos la desigualdad de la condición de descenso suficiente:

$$f(x_0 + p_0) \leq f(x_0) + c_1\alpha \textbf{p}_0^T \nabla f(x_0) $$

$$6 \leq 7+ 10^{-4} \alpha (-2)$$

*despejamos $\alpha$

$$\alpha \leq 20^{4}$$

$\color{lightblue}{Ejercicio \space 4}$ <br>

Sea $f ∶ Rn → R$ y S una matriz no singular de tamaño $n \times n$. Si $x = Sy$ para $y \in Rn$ y definimos
$g(y) = f(Sy)$.

* A plicando la regla de la cadena muestre que $\nabla g(y) = S^{⊤} \nabla f(x)$


Partimos del gradiente de $g(y)$:
$$\nabla _y g(y)= \nabla _y f(Sy)=\nabla _y f(x)$$

$$\implies \nabla _y g(y)= \nabla _x f(x) * \nabla _y x$$

$$\implies \nabla _y g(y)= \nabla _x f(x) * \nabla _y Sy $$

S es una matriz y $ \nabla _y y=(1,1,1,1...)$, por lo que 

$$\nabla _y g(y)= S^{⊤}  \nabla _x f(x)   $$

* Muestre que $−D\nabla𝑓(x_k)$ con $D=SS^{⊤}$ es una dirección de descenso

Partimos del producto entre eñ grandiente y  $ -D\nabla f(x)$



$$\nabla f(x)^{⊤} [-D\nabla f(x)]=-\nabla f(x)^{⊤}  SS^{⊤}  \nabla f(x)$$

$$\implies -[\nabla f(x)^{⊤}  S][S^{⊤}  \nabla f(x) ]$$

$$\implies -[\nabla f(x)  S^{⊤}]^{⊤}[S^{⊤}  \nabla f(x) ]$$


Pero usando que $\nabla _y g(y)= S^{⊤}  \nabla _x f(x) $ (resultado anterior)


$$\implies \nabla f(x)^{⊤} [-D\nabla f(x)]=-[\nabla _y g(y)]^{⊤}[\nabla _y g(y)]=-||\nabla _y g(y)||^2$$

Y la norma siempre es mayor o igual que cero, por lo que 

$$\nabla f(x)^{⊤} [-D\nabla f(x)] \leq 0$$

Así que $-D\nabla f(x)$ es una dirección de descenso